In [9]:
import pandas as pd
data=pd.read_csv('http://bit.ly/kaggletrain')

print(data.isna().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [11]:
data=data.loc[data.Embarked.notna(),['Survived','Pclass','Sex','Embarked']]
print(data.shape)

(889, 4)


In [12]:
X=data.loc[:,'Pclass']
y=data.Survived
print(X.shape,y.shape)

(889,) (889,)


In [22]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [23]:
from sklearn.model_selection import cross_val_score
cross_val_score(logreg, X, y, cv=5, scoring='accuracy').mean()

c:\users\purnota\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\users\purnota\appdata\local\programs\python\python38\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\purnota\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py", line 1342, in fit
    X, y = self._validate_data(X, y, accept_sparse='csr', dtype=_dtype,
  File "c:\users\purnota\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "c:\users\purnota\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py", line 73, in inner_f
    r

nan

In [24]:
y.value_counts(normalize=True)

0    0.617548
1    0.382452
Name: Survived, dtype: float64

In [25]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(sparse=False)

In [27]:
ohe.fit_transform(data[['Sex']])

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [29]:
ohe.categories_

[array(['female', 'male'], dtype=object)]

In [32]:
X=data.drop('Survived',axis='columns')


In [33]:
X.head()

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S


In [34]:
from sklearn.compose import make_column_transformer
col_trans=make_column_transformer((OneHotEncoder(),['Sex','Embarked']),remainder='passthrough')

In [35]:
col_trans.fit_transform(X)

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]])

In [38]:
from sklearn.pipeline import make_pipeline
pipe=make_pipeline(col_trans,logreg)


In [39]:
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

0.7727924839713071

In [40]:
X_new = X.sample(5, random_state=99)
X_new

,Pclass,Sex,Embarked
599,1,male,C
512,1,male,S
273,1,male,C
215,1,female,C
790,3,male,Q


In [41]:
pipe.fit(X,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Sex', 'Embarked'])])),
                ('logisticregression', LogisticRegression())])

In [42]:
pipe.predict(X_new)

array([1, 0, 1, 1, 0], dtype=int64)